In [ ]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import yaml

In [ ]:
cx = 4
cy = 7
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cx*cy,3), np.float32)
objp[:,:2] = np.mgrid[0:cx,0:cy].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('calib_drone/*.jpg')
scs = 0
total = len(images)
checked = 0
for fname in images:
    print(fname)
    checked = checked + 1
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (cx,cy),None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        corners = np.array([[corner for [corner] in corners]])
        scs = scs + 1
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (cx,cy), corners2,ret)
        
    print(fname, checked , '(', scs, ') /', total)    


In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

# transform the matrix and distortion coefficients to writable lists
data = {'camera_matrix': np.asarray(mtx).tolist(),
        'dist_coeff': np.asarray(dist).tolist()}

# and save it to a file
with open("virtual_calibration_matrix.yaml", "w") as f:
    yaml.dump(data, f)

# done

In [ ]:
with open("virtual_calibration_matrix.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)
mtx = np.array(data_loaded['camera_matrix'])
dist = np.array(data_loaded['dist_coeff'])

In [ ]:
img = cv2.imread(np.random.choice(images))
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
plt.figure(figsize=(7,7))
plt.imshow(dst)
plt.figure(figsize=(7,7))
plt.imshow(img)

In [ ]:
newcameramtx